# File IO

> Saving and loading data.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#|default_exp fileio
#|export
import pathlib
import yaml
import json
import pickle
from pathlib import Path
from os import walk
import numpy as np
import csv
from cgnai.utils import listmap, Bunch
import cgnai
Path = pathlib.Path

In [ ]:
#|export
def ls_bash(path="./", absolute=False):
    path = Path(path)
    f = []
    d = []
    for (dirpath, dirnames, filenames) in walk(path):
        dirnames  = listmap(Path, dirnames)
        if absolute:
            filenames = listmap(lambda n: path/n, filenames)
            dirname   = listmap(lambda n: path/n, dirnames)
            
        f.extend(filenames)
        d.extend(dirnames)
        
        break

    return Bunch(path=path, files=sorted(f), dirs=sorted(d), folders=sorted(d))

ls = ls_bash

In [ ]:
ls_bash()

In [ ]:
ls_bash("./", absolute=True)

In [ ]:
#|export
def save(obj, fname):
    """
    Saves an object to a file.
    
    Supported formats are `.yaml`, `.json`, `.npy`, `.npz`
    and everything else will be "pickled".
    """

    p = Path(fname)
    name   = p.stem
    format = p.suffix

    if format==".yaml":
        with open(p, 'w') as f:
            yaml.dump(obj, f,
                allow_unicode=True,
                sort_keys=False,
                indent=4,
                explicit_start=True)

    elif format==".json":
        with open(p, 'w') as f:
            json.dump(obj, f,
                        indent=4)

    elif format==".npy" or format==".npz":
        np.save(p, obj)

    else:
        with open(p, 'wb') as f:
            pickle.dump(obj, f)

In [ ]:
#|export
def dump(obj, fname): 
    """Dumps an object to file.""" 
    return save(obj, fname)

In [ ]:
#|export
def load(fname, format=None):
    
    p = Path(fname)
    name   = p.stem
    
    if format is None:
        format = p.suffix
            
    if format==".yaml":
        with open(p, 'r') as f:
            return yaml.load(f, Loader=yaml.FullLoader)

    elif format==".json":
        with open(p, 'r') as f:
            return json.load(f)

    elif format==".npy" or format==".npz":
        return np.load(p)

    elif format==".txt":
        with open(p, 'r') as f:
            lines = f.read().split('\n')
            return lines
    
    elif format==".pkl":
        with open(p, 'rb') as f:
            return pickle.load(f)
        
    elif format==".tsv":
        with open(p) as f:
            X = csv.reader(f, delimiter="\t", quotechar='"')
            return [x for x in X]
    
    elif format==".csv":
        with open(p) as f:
            X = csv.reader(f, delimiter=",", quotechar='"')            
            return [x for x in X]
                
    else:
        supported = [".yaml", ".json", ".npy", ".npz", ".txt", ".pkl", ".tsv", ".csv"]
        raise ValueError(f"Document type not supported: should be one of the following: \n\t{supported}")
        
    

In [ ]:
arr = np.random.rand(10)
dump(arr, "_temp/numpy.npy")
assert np.any(load("_temp/numpy.npy") == arr, None)
!rm "_temp/numpy.npy"

In [ ]:
def find_config_file(cfg_name=CGNAI_CONFIG):
    cfg_path = Path.cwd()
    while cfg_path != cfg_path.parent and not (cfg_path/cfg_name).exists(): cfg_path = cfg_path.parent
    fname = cfg_path/cfg_name
    return fname

In [ ]:
CGNAI_CONFIG = 'config.yaml'

def get_config(cfg_name=CGNAI_CONFIG):
    cfg_path = Path.cwd()
    while cfg_path != cfg_path.parent and not (cfg_path/cfg_name).exists(): cfg_path = cfg_path.parent
    fname = cfg_path/cfg_name
    return load(fname)

In [ ]:
def set_config(cfg, cfg_name=CGNAI_CONFIG):
    fname = find_config_file(cfg_name)    
    cfg   = save(fname)
    return cfg

In [ ]:
import os
import configparser
def cgnai_config(config_name="config.ini"):
    p = Path(os.path.dirname(os.path.realpath(__file__))) 
    cgnai_wd = p.parents[0]
    c = cgnai_wd/config_name
    
    config = configparser.ConfigParser()    
    try:
        config.read(c)
    except:
        pass
    
    return config
    


In [ ]:
def cgnai_set(nested_key, value, config_name="config.ini"):
    p = Path(os.path.dirname(os.path.realpath(__file__))) 
    cgnaiwd = p.parents[0]
    c = cgnaiwd/config_name
    c.touch()
    
    try:
        section, key = nested_key.split(":")
    except:
        section, key = "OTHER", nested_key

    config = configparser.ConfigParser()
    config.read(c)
    if section not in config: config.add_section(section)
        
    config.set(section, key, value)

    with open(c, "w") as f: config.write(f)

In [ ]:
def cgnai_get(nested_key, cfg_name="config.ini"):
    p = Path(os.path.dirname(os.path.realpath(__file__))) 
    cgnaiwd = p.parents[0]
    c = cgnaiwd/config_name
    try:
        section, key = nested_key.split(":")
    except:
        section, key = "OTHER", nested_key

    config = configparser.ConfigParser()
    config.read(c)
    return config.get(section, key)

In [ ]:
def cgnai_del(nested_key, config_name="config.ini"):
    p = Path(os.path.dirname(os.path.realpath(__file__))) 
    cgnaiwd = p.parents[0]
    c = cgnaiwd/config_name
    
    try:
        section, key = nested_key.split(":")
    except:
        section, key = "OTHER", nested_key
        
        
    
    config = configparser.ConfigParser()
    config.read(c)    
    
    
    if section in config:
        if key in config[section]: 
            del config[section][key]
            
        if len(config[section]) == 0: 
            del config[section]

    with open(c, "w") as f: config.write(f)

In [ ]:
def cgnai_shared(cfg_name="config.ini"):
    """Grabs the path to the shared data foldser from `config.ini`"""
    cfg_path = Path.cwd()
    while cfg_path != cfg_path.parent and not (cfg_path/cfg_name).exists(): cfg_path = cfg_path.parent
    config = configparser.ConfigParser()
    config.read(cfg_path/cfg_name)
    return Path(config["DATA"]["shared"])

In [ ]:
cgnai_shared(cfg_name="config.ini")

In [ ]:
def cgnai_local(cfg_name="config.ini"):
    """Grabs the path to the local data folder from `config.ini`"""
    cfg_path = Path.cwd()
    while cfg_path != cfg_path.parent and not (cfg_path/cfg_name).exists(): cfg_path = cfg_path.parent
    config = configparser.ConfigParser()
    config.read(cfg_path/cfg_name)
    return Path(config["DATA"]["local"])

In [ ]:
cgnai_local(cfg_name="config.ini")